In [2]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(file)
# print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/mnt/c/Users/forfrt/OneDrive/Documents/Research/PhD/Saarland/ART


In [28]:
import re
import random
import pandas as pd

In [121]:
input_file_li=[r"FR_output_features.csv", r"IT_output_features.csv", r"SK_output_features.csv"]
session_li=["main0", "main25", "main50", "main75"]
features=["Mean Pitch", "Max Pitch", "Mean Intensity", "Max Intensity", "Jitter", "Shimmer", "NHR"]

In [122]:
meta_df=pd.read_csv(r"Metadata_IFS.csv")
unique_speakers = meta_df[['spk', 'sex']].drop_duplicates()
unique_speakers.set_index('spk', inplace=False)

speaker_gender_mapping={unique_speakers.loc[ind]['spk']:unique_speakers.loc[ind]['sex'] for ind in unique_speakers.index}

In [143]:
def compute_sentence_proximity(input_df, feature, session, speaker_gender_mapping):
    session_input_df=input_df[input_df['session']==session]
    
    dyad_reg=r"(\d{2})(\d{2})"
    dyad_li=list(set(session_input_df['dyad'].tolist()))
    
    speaker_adj_promixity_dict={}
    partner_adj_promixity_dict={}
    speaker_other_promixity_dict={}
    partner_other_promixity_dict={}

    male_feature_li=list()
    female_feature_li=list()
    speaker_li=list(set(session_input_df['speaker'].tolist()))
    for speaker in speaker_li:
        session_speaker_df=session_input_df[session_input_df['speaker']==speaker]
        feature_li=session_speaker_df[feature].tolist()
        if speaker_gender_mapping[speaker]=='M':
            male_feature_li.extend(feature_li)
            print("MALE speaker, sex", speaker, speaker_gender_mapping[speaker])
        else:
            female_feature_li.extend(feature_li)
            print("FEMALE speaker, sex", speaker, speaker_gender_mapping[speaker])
        
        print("session_speaker_df", session_speaker_df)

    male_feature_mean=sum(male_feature_li)/len(male_feature_li)
    female_feature_mean=sum(female_feature_li)/len(female_feature_li)
    
    for dyad in dyad_li:
        print("processing", dyad)
        match = re.search(dyad_reg, "%04d" % dyad)
        speaker=int(match.group(1))
        partner=int(match.group(2))
    
        session_speaker_df=session_input_df[session_input_df['speaker']==speaker]
        session_partner_df=session_input_df[session_input_df['speaker']==partner]
    
        speaker_sent_li=session_speaker_df['sentence'].tolist()
        partner_sent_li=session_partner_df['sentence'].tolist()
        
        speaker_sent_li.sort()
        partner_sent_li.sort()
        # print("speaker, speaker_sent_li", speaker, speaker_sent_li)
        # print("partner, partner_sent_li", partner, partner_sent_li)
    
        # dialogue_sent_li=speaker_sent_li+partner_sent_li
        # dialogue_sent_li.sort()

        speaker_adj_promixity=speaker_adj_promixity_dict[dyad]=[]
        partner_adj_promixity=partner_adj_promixity_dict[dyad]=[]
        speaker_other_promixity=speaker_other_promixity_dict[dyad]=[]
        partner_other_promixity=partner_other_promixity_dict[dyad]=[]
    
        for id, (speaker_sent, partner_sent)in enumerate(zip(speaker_sent_li, partner_sent_li)):
            # print("id, speaker_sent, partner_sent", id, speaker_sent, partner_sent)
            
            speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==speaker_sent][feature].item()
            partner_feature=session_partner_df.loc[session_partner_df['sentence']==partner_sent][feature].item()
            # print("speaker_feature, partner_feature", speaker_feature, partner_feature)
    
            adj_distance=abs(speaker_feature-partner_feature)
    
            sample_speaker_sent_li=speaker_sent_li.copy()
            sample_partner_sent_li=partner_sent_li.copy()
            sample_speaker_sent_li.remove(speaker_sent)
            sample_partner_sent_li.remove(partner_sent)
            
            random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(sample_speaker_sent_li, 10)]
            random10_partner_features=[session_partner_df.loc[session_partner_df['sentence']==sent][feature].item() for sent in random.sample(sample_partner_sent_li, 10)]
            # print("random10_speaker_features", random10_speaker_features)
            # print("random10_partner_features", random10_partner_features)
            
            other_speaker_distance=abs(speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
            other_partner_distance=abs(partner_feature-sum(random10_speaker_features)/len(random10_speaker_features))
    
            speaker_adj_promixity.append(adj_distance)
            partner_adj_promixity.append(adj_distance)
            speaker_other_promixity.append(other_speaker_distance)
            partner_other_promixity.append(other_partner_distance)
    
            if id<len(speaker_sent_li)-1:
                next_speaker_sent=speaker_sent_li[id+1]
                # print("id, next_speaker_sent", id, next_speaker_sent)
                
                next_speaker_feature=session_speaker_df.loc[session_speaker_df['sentence']==next_speaker_sent][feature].item()
                next_adj_distance=abs(partner_feature-next_speaker_feature)
                
                next_sample_speaker_sent_li=sample_speaker_sent_li.copy()
                next_sample_speaker_sent_li.remove(next_speaker_sent)
                next_random10_speaker_features=[session_speaker_df.loc[session_speaker_df['sentence']==sent][feature].item() for sent in random.sample(next_sample_speaker_sent_li, 10)]
                # print("next_random10_speaker_features", next_random10_speaker_features)
    
                next_other_speaker_distance=abs(next_speaker_feature-sum(random10_partner_features)/len(random10_partner_features))
                next_other_partner_distance=abs(partner_feature-sum(next_random10_speaker_features)/len(next_random10_speaker_features))
    
                speaker_adj_promixity.append(next_adj_distance)
                partner_adj_promixity.append(next_adj_distance)
                speaker_other_promixity.append(next_other_speaker_distance)
                partner_other_promixity.append(next_other_partner_distance)
            
    
    return speaker_adj_promixity_dict, partner_adj_promixity_dict, speaker_other_promixity_dict, partner_other_promixity_dict
        

In [144]:
input_df=pd.read_csv(input_file_li[0])
session=session_li[0]
feature="Max Pitch"

compute_sentence_proximity(input_df, feature, session, speaker_gender_mapping)

FEMALE speaker, sex 30 F
session_speaker_df       Unnamed: 0                   File Name  Mean Pitch   Max Pitch  \
1600        1600  d3031_30_main0_11_mono.wav  127.266650  574.043254   
1601        1601  d3031_30_main0_13_mono.wav   96.111750  278.910715   
1602        1602  d3031_30_main0_15_mono.wav  124.786899  498.109727   
1603        1603  d3031_30_main0_17_mono.wav  122.157421  312.215300   
1604        1604  d3031_30_main0_19_mono.wav  102.846459  280.386654   
1605        1605   d3031_30_main0_1_mono.wav  118.848732  278.786506   
1606        1606  d3031_30_main0_21_mono.wav  128.501805  498.343036   
1607        1607  d3031_30_main0_23_mono.wav  131.284743  572.797450   
1608        1608  d3031_30_main0_25_mono.wav  110.639005  283.814874   
1609        1609  d3031_30_main0_27_mono.wav  125.103915  567.594071   
1610        1610  d3031_30_main0_29_mono.wav  121.068439  455.427093   
1611        1611  d3031_30_main0_31_mono.wav  109.689496  458.979790   
1612        1612  d3

({4647: [153.3455908448438,
   20.168788412657932,
   39.23706588168244,
   78.87098487011599,
   44.35448610490158,
   254.46193430305863,
   288.99127774793027,
   20.42156838796194,
   203.0517081114823,
   4.542120177410425,
   109.21363018626153,
   55.33856288238621,
   236.8306337779615,
   87.55270023399663,
   45.023051737871526,
   35.82932143437688,
   18.01526375362772,
   171.22699830840975,
   194.315004225339,
   18.101735860252745,
   259.50327351719955,
   253.42995921787974,
   25.64025032145537,
   20.566664568095575,
   14.700849230879783,
   14.103595718891142,
   4.475951057133045,
   18.550520227396873,
   20.647125251215243,
   44.566335476947245,
   63.66316058829295,
   274.8624659988753,
   212.88202791634188,
   35.20964225630968,
   83.93191910931478,
   53.82912978095624,
   30.633885495047366,
   204.7530033687882,
   251.28663343904282,
   278.2275445274835,
   305.2830961810565,
   200.7565045893126,
   154.31284117435627,
   272.948573536953,
   305.63

In [81]:
male_gender_mean={}
female_gender_mean={}

for input_file in input_file_li:
    male_file_mean=male_mean[file]=dict()
    female_file_mean=female_mean[file]=dict()
    
    for session in session_li:
        male_file_session_mean=male_file_mean[session]=dict()
        female_file_session_mean=female_file_mean[session]=dict()
        
        input_df=pd.read_csv(input_file)

        t_s_stats=dict()
        t_p_stats=dict()
        p_s_values=dict()
        p_p_values=dict()
        
        for feature in features:
            speaker_adj_promixity_dict, partner_adj_promixity_dict, speaker_other_promixity_dict, partner_other_promixity_dict=compute_sentence_proximity(input_df, feature, session, speaker_gender_mapping)
            dyad_keys=list(speaker_adj_promixity_dict.keys())
        
            t_s_feature_stat=t_s_stats[feature]=dict()
            t_p_feature_stat=t_p_stats[feature]=dict()
        
            p_s_feature_value=p_s_values[feature]=dict()
            p_p_feature_value=p_p_values[feature]=dict()
        
            for k in dyad_keys:
                dyad_output_df=pd.DataFrame(list(zip(speaker_adj_promixity_dict[k], partner_adj_promixity_dict[k], speaker_other_promixity_dict[k], partner_other_promixity_dict[k])), 
                                           columns=['speaker_adj_promixity', 'partner_adj_promixity', 'speaker_other_promixity', 'partner_other_promixity_dict'])
                dyad_output_df.to_csv("SK_{}_{}_promixity.csv".format(k, feature))
        
                t_s_statistic, p_s_value = stats.ttest_rel(speaker_adj_promixity_dict[k], speaker_other_promixity_dict[k])
                t_p_statistic, p_p_value = stats.ttest_rel(partner_adj_promixity_dict[k], partner_other_promixity_dict[k])
        
                t_s_feature_stat[k]=t_s_statistic
                t_p_feature_stat[k]=t_p_statistic
                p_s_feature_value[k]=p_s_value
                p_p_feature_value[k]=p_p_value        
        
            print("t_s_feature_stat, p_s_feature_value", t_s_feature_stat, p_s_feature_value)
            print("t_p_feature_stat, p_p_feature_value", t_p_feature_stat, p_p_feature_value)

                              File Name  Mean Pitch   Max Pitch  \
0     d3031_30_imitation555_10_mono.wav  133.911252  291.467781   
1     d3031_30_imitation555_11_mono.wav  125.613575  299.508872   
2     d3031_30_imitation555_12_mono.wav  106.922285  579.759460   
3     d3031_30_imitation555_13_mono.wav  146.255313  348.450336   
4     d3031_30_imitation555_14_mono.wav  133.373945  285.964270   
...                                 ...         ...         ...   
6395       d5051_51_solo555_79_mono.wav   68.554602  243.799618   
6396        d5051_51_solo555_7_mono.wav   78.350545  484.437188   
6397       d5051_51_solo555_80_mono.wav   85.254687  211.843897   
6398        d5051_51_solo555_8_mono.wav   67.478382  489.392877   
6399        d5051_51_solo555_9_mono.wav   75.833070  567.421015   

      Mean Intensity  Max Intensity    Jitter   Shimmer        NHR  
0          68.730600      77.863271  0.013146  0.058427  16.358906  
1          69.673192      79.817015  0.018806  0.067646  

In [66]:
t_s_stats=dict()
t_p_stats=dict()
p_s_values=dict()
p_p_values=dict()

for feature in features:
    speaker_adj_promixity_dict, partner_adj_promixity_dict, speaker_other_promixity_dict, partner_other_promixity_dict=compute_sentence_proximity(input_df, feature, session)
    dyad_keys=list(speaker_adj_promixity_dict.keys())

    t_s_feature_stat=t_s_stats[feature]=dict()
    t_p_feature_stat=t_p_stats[feature]=dict()

    p_s_feature_value=p_s_values[feature]=dict()
    p_p_feature_value=p_p_values[feature]=dict()

    for k in dyad_keys:
        dyad_output_df=pd.DataFrame(list(zip(speaker_adj_promixity_dict[k], partner_adj_promixity_dict[k], speaker_other_promixity_dict[k], partner_other_promixity_dict[k])), 
                                   columns=['speaker_adj_promixity', 'partner_adj_promixity', 'speaker_other_promixity', 'partner_other_promixity_dict'])
        dyad_output_df.to_csv("SK_{}_{}_promixity.csv".format(k, feature))

        t_s_statistic, p_s_value = stats.ttest_rel(speaker_adj_promixity_dict[k], speaker_other_promixity_dict[k])
        t_p_statistic, p_p_value = stats.ttest_rel(partner_adj_promixity_dict[k], partner_other_promixity_dict[k])

        t_s_feature_stat[k]=t_s_statistic
        t_p_feature_stat[k]=t_p_statistic
        p_s_feature_value[k]=p_s_value
        p_p_feature_value[k]=p_p_value        

    print("t_s_feature_stat, p_s_feature_value", t_s_feature_stat, p_s_feature_value)
    print("t_p_feature_stat, p_p_feature_value", t_p_feature_stat, p_p_feature_value)

processing 4647
processing 4041
processing 3435
processing 4849
processing 4243
processing 3637
processing 3031
processing 5051
processing 4445
processing 3839
t_s_feature_stat, p_s_feature_value {4647: 2.290550254506784, 4041: 1.1910643535873626, 3435: 3.751055645828282, 4849: 0.9063895633331994, 4243: 2.7779118410942294, 3637: 1.9357754287838165, 3031: 0.5208107478786659, 5051: 2.488681418039935, 4445: 1.7276309240569838, 3839: 0.7209553905720596} {4647: 0.02469096023638453, 4041: 0.23724022538027553, 3435: 0.00033668828801578204, 4849: 0.3675212276894134, 4243: 0.006850065907542629, 3637: 0.056518711750795814, 3031: 0.6039740044495923, 5051: 0.014952227328558089, 4445: 0.0880120797474087, 3839: 0.4730923148053119}
t_p_feature_stat, p_p_feature_value {4647: 4.258752208963678, 4041: 1.5864217945311456, 3435: 3.6701908571342483, 4849: 0.19806696626309295, 4243: 2.169527570109056, 3637: 0.3073397688913997, 3031: 0.8733240322466344, 5051: 4.7580117197721075, 4445: 1.0543510804737102, 383

In [69]:
t_p_stats_df = pd.DataFrame(t_p_stats)
t_s_stats_df = pd.DataFrame(t_s_stats)
p_s_values_df = pd.DataFrame(p_s_values)
p_p_values_df = pd.DataFrame(p_p_values)

p_s_values_df=p_s_values_df.rename(columns={"Mean Pitch":"Mean Pitch(p)", "Max Pitch":"Max Pitch(p)", "Mean Intensity":"Mean Intensity(p)", 
                              "Max Intensity":"Max Intensity(p)", "Jitter":"Jitter(p)", "Shimmer":"Shimmer(p)", "NHR":"NHR(p)"})
p_p_values_df=p_p_values_df.rename(columns={"Mean Pitch":"Mean Pitch(p)", "Max Pitch":"Max Pitch(p)", "Mean Intensity":"Mean Intensity(p)",
                              "Max Intensity":"Max Intensity(p)", "Jitter":"Jitter(p)", "Shimmer":"Shimmer(p)", "NHR":"NHR(p)"})


,Mean Pitch(p),Max Pitch(p),Mean Intensity(p),Max Intensity(p),Jitter(p),Shimmer(p),NHR(p)
4647,0.000057,0.141040,0.635421,0.074217,0.525912,0.904426,0.547107
4041,0.116690,0.316835,0.637751,0.989717,0.004439,0.141635,0.288632
3435,0.000442,0.213223,0.740359,0.973753,0.068300,0.277468,0.476773
4849,0.843508,0.432261,0.737915,0.505980,0.127392,0.814995,0.593403
4243,0.033088,0.728188,0.493534,0.852533,0.096378,0.004415,0.023553
3637,0.759404,0.000047,0.006375,0.000001,0.005021,0.040782,0.304242
3031,0.385167,0.032179,0.890932,0.952825,0.000076,0.022450,0.103299
5051,0.000009,0.047355,0.000024,0.000158,0.135179,0.000105,0.000056
4445,0.294977,0.727495,0.008193,0.039216,0.060723,0.684418,0.026450
3839,0.012223,0.086382,0.592360,0.603697,0.974045,0.959261,0.000262


In [72]:
t_all_stats_df = t_p_stats_df.append(t_s_stats_df)
p_all_values_df = p_s_values_df.append(p_p_values_df)

p_all_values_df

,Mean Pitch(p),Max Pitch(p),Mean Intensity(p),Max Intensity(p),Jitter(p),Shimmer(p),NHR(p)
4647,0.024691,0.018707,2.593350e-01,4.829991e-01,0.332225,0.840515,0.950786
4041,0.237240,0.900376,9.147930e-01,8.848327e-01,0.000020,0.008837,0.150323
3435,0.000337,0.247694,5.090069e-01,8.579782e-01,0.000006,0.293541,0.089362
4849,0.367521,0.884447,8.765159e-01,6.051790e-01,0.009249,0.497771,0.668169
4243,0.006850,0.004589,9.691714e-01,7.779716e-01,0.169905,0.000270,0.194536
3637,0.056519,0.677867,4.214208e-02,3.136310e-07,0.034680,0.156055,0.684022
3031,0.603974,0.868406,5.673689e-01,8.618711e-01,0.000991,0.000220,0.007258
5051,0.014952,0.000005,3.277671e-04,6.786870e-06,0.012761,0.005590,0.002130
4445,0.088012,0.881894,2.867434e-21,1.644650e-10,0.010557,0.847208,0.000756
3839,0.473092,0.006577,7.256846e-01,9.977820e-01,0.368583,0.884925,0.000019


In [73]:
all_values_df=pd.merge(t_all_stats_df, p_all_values_df, left_index=True, right_index=True)
all_values_df

,Mean Pitch,Max Pitch,Mean Intensity,Max Intensity,Jitter,Shimmer,NHR,Mean Pitch(p),Max Pitch(p),Mean Intensity(p),Max Intensity(p),Jitter(p),Shimmer(p),NHR(p)
3031,0.873324,2.181223,-0.137574,0.059350,4.176849,2.328994,1.648371,0.603974,0.868406,5.673689e-01,8.618711e-01,0.000991,0.000220,0.007258
3031,0.873324,2.181223,-0.137574,0.059350,4.176849,2.328994,1.648371,0.385167,0.032179,8.909318e-01,9.528249e-01,0.000076,0.022450,0.103299
3031,0.520811,0.166230,0.574372,-0.174568,3.422396,3.875456,2.757162,0.603974,0.868406,5.673689e-01,8.618711e-01,0.000991,0.000220,0.007258
3031,0.520811,0.166230,0.574372,-0.174568,3.422396,3.875456,2.757162,0.385167,0.032179,8.909318e-01,9.528249e-01,0.000076,0.022450,0.103299
3435,3.670191,1.255007,-0.332559,0.033007,1.848633,1.093666,0.714953,0.000337,0.247694,5.090069e-01,8.579782e-01,0.000006,0.293541,0.089362
3435,3.670191,1.255007,-0.332559,0.033007,1.848633,1.093666,0.714953,0.000442,0.213223,7.403591e-01,9.737534e-01,0.068300,0.277468,0.476773
3435,3.751056,1.164690,-0.663437,0.179541,4.855423,1.057514,1.720199,0.000337,0.247694,5.090069e-01,8.579782e-01,0.000006,0.293541,0.089362
3435,3.751056,1.164690,-0.663437,0.179541,4.855423,1.057514,1.720199,0.000442,0.213223,7.403591e-01,9.737534e-01,0.068300,0.277468,0.476773
3637,0.307340,4.309316,2.803571,5.247125,2.887615,2.080298,1.034192,0.056519,0.677867,4.214208e-02,3.136310e-07,0.034680,0.156055,0.684022
3637,0.307340,4.309316,2.803571,5.247125,2.887615,2.080298,1.034192,0.759404,0.000047,6.374505e-03,1.293273e-06,0.005021,0.040782,0.304242


In [78]:
cols=['Mean Pitch', 'Mean Pitch(p)', 'Max Pitch', 'Max Pitch(p)', 'Mean Intensity', 'Mean Intensity(p)',
      'Max Intensity', 'Max Intensity(p)','Jitter', 'Jitter(p)','Shimmer', 'Shimmer(p)', 'NHR', 'NHR(p)']
all_values_df=all_values_df[cols]
all_values_df.to_csv("FR_promixity.csv")

In [79]:
all_values_df

,Mean Pitch,Mean Pitch(p),Max Pitch,Max Pitch(p),Mean Intensity,Mean Intensity(p),Max Intensity,Max Intensity(p),Jitter,Jitter(p),Shimmer,Shimmer(p),NHR,NHR(p)
3031,0.873324,0.603974,2.181223,0.868406,-0.137574,5.673689e-01,0.059350,8.618711e-01,4.176849,0.000991,2.328994,0.000220,1.648371,0.007258
3031,0.873324,0.385167,2.181223,0.032179,-0.137574,8.909318e-01,0.059350,9.528249e-01,4.176849,0.000076,2.328994,0.022450,1.648371,0.103299
3031,0.520811,0.603974,0.166230,0.868406,0.574372,5.673689e-01,-0.174568,8.618711e-01,3.422396,0.000991,3.875456,0.000220,2.757162,0.007258
3031,0.520811,0.385167,0.166230,0.032179,0.574372,8.909318e-01,-0.174568,9.528249e-01,3.422396,0.000076,3.875456,0.022450,2.757162,0.103299
3435,3.670191,0.000337,1.255007,0.247694,-0.332559,5.090069e-01,0.033007,8.579782e-01,1.848633,0.000006,1.093666,0.293541,0.714953,0.089362
3435,3.670191,0.000442,1.255007,0.213223,-0.332559,7.403591e-01,0.033007,9.737534e-01,1.848633,0.068300,1.093666,0.277468,0.714953,0.476773
3435,3.751056,0.000337,1.164690,0.247694,-0.663437,5.090069e-01,0.179541,8.579782e-01,4.855423,0.000006,1.057514,0.293541,1.720199,0.089362
3435,3.751056,0.000442,1.164690,0.213223,-0.663437,7.403591e-01,0.179541,9.737534e-01,4.855423,0.068300,1.057514,0.277468,1.720199,0.476773
3637,0.307340,0.056519,4.309316,0.677867,2.803571,4.214208e-02,5.247125,3.136310e-07,2.887615,0.034680,2.080298,0.156055,1.034192,0.684022
3637,0.307340,0.759404,4.309316,0.000047,2.803571,6.374505e-03,5.247125,1.293273e-06,2.887615,0.005021,2.080298,0.040782,1.034192,0.304242


In [ ]:
Metadata_IFS